<a href="https://colab.research.google.com/github/induxtrias/gif-word-neon/blob/main/gif_neon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install pillow ipywidgets

import ipywidgets as widgets
from IPython.display import display, Image as IPImage
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import io

class GifTextApp:
    def __init__(self):
        self.text_input = widgets.Text(
            value='',
            placeholder='Ingresa las palabras separadas por espacios',
            description='Texto:',
            disabled=False,
            layout=widgets.Layout(width='500px')
        )

        self.font_size_input = widgets.IntSlider(
            value=40,
            min=10,
            max=100,
            step=1,
            description='Tamaño de fuente:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )

        self.color_picker = widgets.ColorPicker(
            concise=False,
            description='Color neón:',
            value='#0000FF',
            disabled=False
        )

        self.animation_speed = widgets.FloatSlider(
            value=1.0,
            min=0.5,
            max=2.0,
            step=0.1,
            description='Velocidad:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )

        self.create_button = widgets.Button(
            description='Crear GIF',
            disabled=False,
            button_style='',
            tooltip='Haz clic para crear el GIF',
            icon='check'
        )
        self.create_button.on_click(self.create_gif)

        self.output = widgets.Output()

        display(self.text_input)
        display(widgets.HBox([self.font_size_input, self.color_picker, self.animation_speed]))
        display(self.create_button)
        display(self.output)

    def create_gif(self, b):
        with self.output:
            self.output.clear_output()
            text = self.text_input.value.split()[:50]  # Limitar a 50 palabras
            if not text:
                print("Por favor, ingresa algún texto.")
                return

            print("Creando GIF... Por favor espera.")
            images = []
            for word in text:
                word = word.strip()
                for brightness in [1.0, 1.5, 2.0, 2.5, 3.0, 2.5, 2.0, 1.5]:
                    img = self.create_neon_image(word, brightness)
                    images.append(img)

            gif_path = "neon_text_animation.gif"
            images[0].save(gif_path, save_all=True, append_images=images[1:], loop=0,
                           duration=int(100/self.animation_speed.value))
            display(IPImage(filename=gif_path))
            print(f"GIF creado: {gif_path}")

    def create_neon_image(self, text, brightness):
        img_width, img_height = 500, 300
        img = Image.new('RGB', (img_width, img_height), color=(0, 0, 0))
        draw = ImageDraw.Draw(img)

        try:
            font = ImageFont.truetype("arial.ttf", self.font_size_input.value)
        except IOError:
            font = ImageFont.load_default()

        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        text_x = (img_width - text_width) / 2
        text_y = (img_height - text_height) / 2

        neon_color = tuple(int(self.color_picker.value.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))

        # Dibujar texto con efecto neón
        for offset in range(3, 0, -1):
            draw.text((text_x-offset, text_y-offset), text, font=font,
                      fill=tuple(int(c * (1 - offset/4)) for c in neon_color))
        draw.text((text_x, text_y), text, font=font, fill=neon_color)

        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(brightness)

        return img

# Crear y mostrar la aplicación
app = GifTextApp()


Text(value='', description='Texto:', layout=Layout(width='500px'), placeholder='Ingresa las palabras separadas…

Button(description='Crear GIF', icon='check', style=ButtonStyle(), tooltip='Haz clic para crear el GIF')

Output()